In [1]:
from qiskit_experiments.library import RoughDragCal
from qiskit_experiments.calibration_management import (
    Calibrations,
    FixedFrequencyTransmon,
)
from qiskit_experiments.data_processing.data_processor import DataProcessor
from qiskit.providers.fake_provider import FakePerth

# replace this lines with an IBM Quantum backend to run the experiment.
backend = FakePerth()
cals = Calibrations.from_backend(backend, libraries=[FixedFrequencyTransmon()])

# Define the experiment
qubit = 2
cal_drag = RoughDragCal(qubit, cals, schedule_name='sx', backend=backend)

# Enable restless measurements by setting the run options and data processor
cal_drag.enable_restless(rep_delay=1e-6)

print(cal_drag.analysis.options.data_processor)
print(cal_drag.run_options)

DataProcessor(input_key=memory, nodes=[RestlessToCounts(validate=True), Probability(validate=True, outcome=1, alpha_prior=[0.5, 0.5])])
Options(meas_level=<MeasLevel.CLASSIFIED: 2>, rep_delay=1e-06, init_qubits=False, memory=True, meas_return=<MeasReturnType.SINGLE: 'single'>, use_measure_esp=False)


In [ ]:
drag_data_restless = cal_drag.run()

In [2]:
from qiskit_experiments.data_processing import (
    DataProcessor,
    Probability,
)

# define a standard data processor.
standard_processor = DataProcessor("counts", [Probability("1")])

cal_drag = RoughDragCal(qubit, cals, schedule_name='sx', backend=backend)
cal_drag.analysis.set_options(data_processor=standard_processor)

# enable restless mode and set override_processor_by_restless to False.
cal_drag.enable_restless(rep_delay=1e-6, override_processor_by_restless=False)

In [3]:
from qiskit import schedule, transpile
from qiskit_experiments.framework import BackendData

dt = BackendData(backend).dt
inst_map = backend.instruction_schedule_map
meas_length = inst_map.get("measure", (qubit, )).duration * dt

# Compute the average duration of all circuits
# Remove measurement instructions
circuits = []
for qc in cal_drag.circuits():
    qc.remove_final_measurements(inplace=True)
    circuits.append(qc)

# Schedule the circuits to obtain the duration of all the gates
executed_circs = transpile(
    circuits,
    backend,
    initial_layout=[qubit],
    scheduling_method="alap",
    **cal_drag.transpile_options.__dict__,
)
durations = [c.duration for c in executed_circs]

tau = sum(durations) * dt / (len(durations))

n_circs = len(cal_drag.circuits())
# can be obtained from backend.default_rep_delay on a backend from qiskit-ibm-provider

delay_s = 0.0025
delay_r = 1e-6  # restless delay
reset = 4e-6  # Estimated reset duration
speed_up = (meas_length + reset + delay_s + tau) / (meas_length + delay_r + tau)
print(f"The QPU will spend {speed_up:.1f}x less time running restless Drag.")

The QPU will spend 1326.1x less time running restless Drag.
